In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
!ls

acoustic.png  catboost_info  data  EarthQuake_One.ipynb  ttf.png


In [3]:
%%time
# train_df = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
train_df = pd.read_csv('data/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

CPU times: user 1min 39s, sys: 5.73 s, total: 1min 45s
Wall time: 1min 52s


In [9]:
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

In [10]:
rows = 15_000
segments = int(np.floor(train_df.shape[0] / rows))

X_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min', 'mad', 'kur', 'skew', 'q95', 'q99','q05', 'q01'])
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

for segment in tqdm(range(segments)):
    seg = train_df.iloc[segment*rows:segment*rows+rows]
    x = seg['acoustic_data']
    y = seg['time_to_failure'].values[-1]
    
    y_train.loc[segment, 'time_to_failure'] = y
    
    X_train.loc[segment, 'ave'] = x.mean()
    X_train.loc[segment, 'std'] = x.std()
    X_train.loc[segment, 'max'] = x.max()
    X_train.loc[segment, 'min'] = x.min()
    X_train.loc[segment, 'mad'] = x.mad()
    X_train.loc[segment, 'kur'] = x.kurtosis()
    X_train.loc[segment, 'skew'] = x.skew()
    X_train.loc[segment, 'q95'] = np.quantile(x,0.95)
    X_train.loc[segment, 'q99'] = np.quantile(x,0.99)
    X_train.loc[segment, 'q05'] = np.quantile(x,0.05)
    X_train.loc[segment, 'q01'] = np.quantile(x,0.01)

100%|██████████| 24534/24534 [02:49<00:00, 145.10it/s]


In [11]:
X_train.head(10)

,ave,std,max,min,mad,kur,skew,q95,q99,q05,q01
0,5.083667,9.648327,104.0,-98.0,5.293493,21.406174,-0.131321,16.0,36.0,-6.0,-27.0
1,4.939733,3.933370,30.0,-20.0,2.849929,4.195286,-0.039745,11.0,15.0,-1.0,-5.0
2,4.931600,5.220505,40.0,-35.0,3.739596,4.026044,0.000936,13.0,19.0,-3.0,-10.0
3,4.761800,3.950776,34.0,-19.0,2.930669,3.339573,0.176724,11.0,15.0,-1.0,-6.0
4,5.143667,5.301783,52.0,-56.0,3.419978,14.269695,-0.055047,12.0,21.0,-2.0,-10.0
5,5.120533,4.549511,52.0,-37.0,2.987598,15.897462,0.145959,11.0,17.0,-1.0,-7.0
6,5.046667,4.279078,40.0,-32.0,2.986257,8.247734,-0.003798,11.0,16.0,-1.0,-6.0
7,4.797333,3.580342,30.0,-21.0,2.722803,2.220999,0.010936,10.0,14.0,-1.0,-4.0
8,4.483467,3.244117,22.0,-16.0,2.516483,1.387338,-0.023910,10.0,12.0,-1.0,-3.0
9,4.532667,4.224041,26.0,-16.0,3.163073,1.819901,0.071072,11.0,16.0,-2.0,-7.0


In [12]:
y_train.head(10)

,time_to_failure
0,1.465897
1,1.461699
2,1.458496
3,1.454197
4,1.449999
5,1.446796
6,1.442497
7,1.438299
8,1.435096
9,1.430797


In [ ]:
X_train.shape,y_train.shape

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [ ]:
X_train_scaled.shape

In [ ]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 50)
rf.fit(X_train_scaled, y_train.values.flatten())
y_predrf = rf.predict(X_train_scaled)

In [ ]:
m = CatBoostRegressor(loss_function='MAE')
m.fit(X_train_scaled, y_train.values.flatten(), silent=True)
y_predcb = m.predict(X_train_scaled)

In [ ]:
# plt.figure(figsize=(6, 6))
# plt.scatter(y_train.values.flatten(), y_pred)
# plt.xlim(0, 20)
# plt.ylim(0, 20)
# plt.xlabel('actual', fontsize=12)
# plt.ylabel('predicted', fontsize=12)
# plt.plot([(0, 0), (20, 20)], [(0, 0), (20, 20)])
# plt.show()

In [ ]:
scorerf = mean_absolute_error(y_train.values.flatten(), y_predrf)
print(f'Scorerf: {scorerf:0.3f}')
scorecb = mean_absolute_error(y_train.values.flatten(), y_predcb)
print(f'Scorecb: {scorecb:0.3f}')


In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')

In [ ]:
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [ ]:
# X_test.head(10)
for seg_id in X_test.index:
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    
    x = seg['acoustic_data']
    
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
    X_test.loc[segment, 'mad'] = x.mad()
    X_test.loc[segment, 'kur'] = x.kurtosis()
    X_test.loc[segment, 'skew'] = x.skew()
    X_test.loc[segment, 'q95'] = np.quantile(x,0.95)
    X_test.loc[segment, 'q99'] = np.quantile(x,0.99)
    X_test.loc[segment, 'q05'] = np.quantile(x,0.05)
    X_test.loc[segment, 'q01'] = np.quantile(x,0.01)

In [ ]:
X_test_scaled = scaler.transform(X_test)

subrf = rf.predict(X_test_scaled)
subcb = m.predict(X_test_scaled)
submission['time_to_failure'] = (subrf + subcb) / 2
submission.to_csv('sub220119.csv')